In [1]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.6/653.6 kB 63.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 30.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from ktrain.text.zsl import ZeroShotClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Find the first category from the prpbability

In [4]:
def find_max_categories_for_lists(lists_of_category_scores):
    max_categories = []
    for category_scores in lists_of_category_scores:
        max_category = max(category_scores, key=lambda x: x[1])[0]
        max_categories.append(max_category)
    return max_categories

# Find the maximum first 2 categories fro mthe probability

In [5]:
def find_max_2_categories_for_lists(lists_of_category_scores):
    max_categories = []
    for category_scores in lists_of_category_scores:
        # Sort the categories based on their values in descending order
        sorted_categories = sorted(category_scores, key=lambda x: x[1], reverse=True)
        # Select the top two categories
        top_categories = [category[0] for category in sorted_categories[:2]]
        max_categories.append(top_categories)
    return max_categories

## check for the best category from the first 2 categories depending on the label of this data
if any of the 2 values are equal to the true label then return it


if none of them are equal to the true label then return the first value with the highest probability

In [6]:
def return_best_category_from_2_categories(list1, list2):
  result = []
  for item1, item2 in zip(list1, list2):
      found = False
      for value in item2:
          if value in item1:
              result.append(value)
              found = True
              break
      if not found:
          result.append(item2[0])
  return result


In [7]:
def compute_classification_report(y_true, y_pred):

    report = classification_report(y_true, y_pred)
    print(report)
    accuracy = accuracy_score(y_true, y_pred)
    return accuracy

In [8]:
def find_max_value(category_scores):
    max_value = max(category_scores, key=lambda x: x[1])[1]
    return max_value

In [9]:
df2 = pd.read_csv("https://github.com/mhjabreel/CharCnn_Keras/raw/master/data/ag_news_csv/train.csv")
df2.columns = ["label" , "title" , "content"]
df2['label'] = df2['label'].map({1: "World", 2: "Sports", 3: "Business", 4: "Sci/Tech"})
df2

,label,title,content
0,Business,Carlyle Looks Toward Commercial Aerospace (Reu...,Reuters - Private investment firm Carlyle Grou...
1,Business,Oil and Economy Cloud Stocks' Outlook (Reuters),Reuters - Soaring crude prices plus worries\ab...
2,Business,Iraq Halts Oil Exports from Main Southern Pipe...,Reuters - Authorities have halted oil export\f...
3,Business,"Oil prices soar to all-time record, posing new...","AFP - Tearaway world oil prices, toppling reco..."
4,Business,"Stocks End Up, But Near Year Lows (Reuters)",Reuters - Stocks ended slightly higher on Frid...
...,...,...,...
119994,World,Pakistan's Musharraf Says Won't Quit as Army C...,KARACHI (Reuters) - Pakistani President Perve...
119995,Sports,Renteria signing a top-shelf deal,Red Sox general manager Theo Epstein acknowled...
119996,Sports,Saban not going to Dolphins yet,The Miami Dolphins will put their courtship of...
119997,Sports,Today's NFL games,PITTSBURGH at NY GIANTS Time: 1:30 p.m. Line: ...


In [16]:
X = df2.content.values
X2 = df2.title.values
Y = df2.label.values

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X,Y , shuffle=True, test_size = 200, random_state = 42 , stratify=Y)

In [13]:
zsl = ZeroShotClassifier()

In [18]:
%%time
df2_prb = zsl.predict(X_test, labels=["World","Sports","Business","Sci/Tech"], include_labels=True)

/usr/local/lib/python3.10/dist-packages/ktrain/text/zsl/core.py:101: UserWarning: TIP: Try increasing batch_size to speedup ZeroShotClassifier predictions
  warnings.warn(


CPU times: user 11.9 s, sys: 79.5 ms, total: 11.9 s
Wall time: 14.1 s


In [19]:
df2_pred = find_max_categories_for_lists(df2_prb)

In [20]:
compute_classification_report(y_test , df2_pred)

              precision    recall  f1-score   support

    Business       0.59      0.72      0.65        50
    Sci/Tech       0.78      0.42      0.55        50
      Sports       0.96      0.98      0.97        50
       World       0.70      0.86      0.77        50

    accuracy                           0.74       200
   macro avg       0.76      0.74      0.73       200
weighted avg       0.76      0.74      0.73       200



0.745

# After getting the 2 maximum probability categories i check if **the first or the second** category is the same as true label and get the classification report

In [21]:
df2_pred = find_max_2_categories_for_lists(df2_prb)

In [23]:
df2_pred = return_best_category_from_2_categories(y_test , df2_pred)

In [25]:
compute_classification_report(y_test , df2_pred)

              precision    recall  f1-score   support

    Business       0.79      0.90      0.84        50
    Sci/Tech       0.95      0.74      0.83        50
      Sports       1.00      1.00      1.00        50
       World       0.91      0.98      0.94        50

    accuracy                           0.91       200
   macro avg       0.91      0.91      0.90       200
weighted avg       0.91      0.91      0.90       200



0.905

# **1111111111111111111111111111**

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X2,Y , shuffle=True, test_size = 200, random_state = 42 , stratify=Y)

In [27]:
%%time
df2_prb = zsl.predict(X_test, labels=["World","Sports","Business","Sci/Tech"], include_labels=True)

/usr/local/lib/python3.10/dist-packages/ktrain/text/zsl/core.py:101: UserWarning: TIP: Try increasing batch_size to speedup ZeroShotClassifier predictions
  warnings.warn(


CPU times: user 5.49 s, sys: 46.5 ms, total: 5.54 s
Wall time: 8.42 s


In [28]:
df2_pred = find_max_categories_for_lists(df2_prb)

In [29]:
compute_classification_report(y_test , df2_pred)

              precision    recall  f1-score   support

    Business       0.48      0.72      0.58        50
    Sci/Tech       0.72      0.26      0.38        50
      Sports       0.88      0.72      0.79        50
       World       0.61      0.80      0.69        50

    accuracy                           0.62       200
   macro avg       0.67      0.62      0.61       200
weighted avg       0.67      0.62      0.61       200



0.625

# After getting the 2 maximum probability categories i check if **the first or the second** category is the same as true label and get the classification report

In [30]:
df2_pred = find_max_2_categories_for_lists(df2_prb)

In [31]:
df2_pred = return_best_category_from_2_categories(y_test , df2_pred)

In [32]:
compute_classification_report(y_test , df2_pred)

              precision    recall  f1-score   support

    Business       0.70      0.90      0.79        50
    Sci/Tech       0.90      0.56      0.69        50
      Sports       0.91      0.80      0.85        50
       World       0.79      0.96      0.86        50

    accuracy                           0.81       200
   macro avg       0.83      0.80      0.80       200
weighted avg       0.83      0.81      0.80       200



0.805